We will create 3 different models:
- PD (Probability of Default) - showing a customer not repaying their debt in full or on time.
- LGD (Loss Given Default) - showing the proportion of an exposure a company loses when a customer defaults.
The proportion of the total exposure that cannot be recovered by the lender once a default has occured.
- EAD (Expoosure at Default) - showing the total loss in terms of amount of money the bank is exposed to if a customer defaults.

EL (expected loss) = PD * LGD * EAD

Estimating expected loss (expected credit loss)
Definition: The amount a lender might lose by lending to a borrower

Basel II Accord prescribes that regulators should allow banks to choose from 3 different approaches for calculating or modeling credit risk (calculating or modeling each of the 3 components of the expected loss):
- standardized approach (SA)
- internal ratings based (IRB) approaches
    - foundation internal ratings based (F-IRB) approach
    - advanced internal ratings based (A-IRB) approach

In this project, we are going to focus on the general case:
Enough data are available for implementing a traditional statistical methodology to build credit risk models.

Loan to value ratio (LTV) : The proportion of the property that is funded by the loan

Return on assets (ROA) = Net income / Total assets
Return on equity (ROE) = Net income / Shareholders' equity
Current ratio = Current assets / Current liabilities
Debt ratio = Total liabilities / Total assets

Application model : is used to estimate a firm's credit rating at the moment of application
The estimated credit ratings are the basis oon which banks decide whether to grant a loan or not, a bank may also use the estimated credit ratings to decide how to price the loan.

Behavioral model: is used to calculate probability of default and respectively expected loss after a loan is granted, banks may also use behavioral models to decide whether to grant an additional loan to an existing customer.


Our goal is to learn how to build statistical models for estimating EL.